In [ ]:
# Our numerical workhorse
import numpy as np
import scipy.special
import pandas as pd

# Import pyplot for plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Beeswarm plots
import beeswarm as bs

# Seaborn, useful for graphics
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

In [ ]:
# load the data
df_aer = pd.read_csv('data/chure/20151001_aerobic_mean_intensities.csv', delimiter=',', comment='#')
df_anaer = pd.read_csv('data/chure/20151001_aerobic_mean_intensities.csv', delimiter=',', comment='#')

#take a look at the data
df_aer.head()

In [ ]:
df_anaer.head()

In [ ]:
# Tidy data
df_aer.columns = (0, 1e1, 1e2, 1e3, 1e4, 1e5)
df_aer = pd.melt(df_aer, var_name='plasmids/cell', value_name='mean pixel intensity').dropna()
df_aer['aerobic'] = True

df_anaer.columns = (0, 1e1, 1e2, 1e3, 1e4, 1e5)
df_anaer = pd.melt(df_anaer, var_name='plasmids/cell', value_name='mean pixel intensity').dropna()
df_anaer['aerobic'] = False

df = df_aer
df = df.append(df_anaer)

In [ ]:
# Check that we appended properly
df[df.aerobic==False].head()

In [ ]:
# Initial plots to get an idea what data looks like
data_to_plot = [df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==0)],
                df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==1e1)],
                df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==1e2)],
                df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==1e3)],
                df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==1e4)],
                df['mean pixel intensity'][(df.aerobic==False) & (df['plasmids/cell']==1e5)],]
plt.boxplot(data_to_plot)
plt.ylim((0, 600))

In [ ]:
# plt.scatter(df['mean pixel intensity'])

In [ ]:
# _ = bs.beeswarm(data_to_plot)
# plt.grid(axis='x')
# plt.ylabel('$N_A$ (min)')
# plt.ylim((0, 60))

Problem 3.1a
For each data set, compute the number of of transformed cells. You will have to decide on a criterion for what constitutes a transformed cell. Comment on how you decided on this criterion.
We have two options:
* A cell is transformed when it has an altered phenotype.
* A cell is transformed when it has an altered genotype.
In my opinion, it makes most biological sense for us to define a cell as transformed when its phenotype is altered at least one plasmid. We see from the initial plot of the data that the cells only really start to change phenotype (expressing enough YFP to change the mean pixel intensity) after $~10^4$ plasmids are introduced into the cell. However, if the cell has accepted at least one plasmid 